In [1]:
import pandas as pd
import numpy as np
import os

---

In [2]:
fullContests = pd.read_csv('data/WorkingData.csv')
durations = pd.read_csv('data/Durations.csv').drop(columns=['Unnamed: 0'])

In [3]:
def combineChunkPreds(chunkName):
    dirList = os.listdir("data/results/"+chunkName+"/")
    chunkResults = []
    for filename in dirList[:]: #limit to .csv
        if(filename[-4:] == '.csv'):
            version = filename.split('_')[2]
            load = pd.read_csv('data/results/'+chunkName+'/'+filename).drop(columns=['Unnamed: 0'])
            load.columns = ['ContestId', version, "label"]
            load = load.drop(columns=['label'])
            chunkResults.append(load)
    combinedDF = chunkResults[0]
    for nextDF in chunkResults[1:]:
        combinedDF = pd.merge(combinedDF, nextDF, on='ContestId', how='left')
    return combinedDF


In [20]:
dirList = os.listdir("data/results/")
kalmanResults = []
for filename in dirList: #limit to .csv
    if(filename[0] != '.' and filename[0] != '_' ):
#         print(filename)
        r = combineChunkPreds(filename)
        kalmanResults.append(r)
#         kalmanResults.append(pd.read_csv('data/results/'+filename).drop(columns=['Unnamed: 0']))
resultsDF = pd.concat(kalmanResults)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  if __name__ == '__main__':


In [36]:
resultsDF = resultsDF.dropna(axis=1)

In [37]:
versions = []
for v in resultsDF.columns:
    if(not("_" in v or v=='ContestId')):
        versions.append(v)
    

In [39]:
cids = resultsDF['ContestId'].unique()
combinedObjs = []
count = 0
for cid in cids[:]:
    obj = {'ContestId':cid}
    count+=1
    duration = durations[durations['ContestId']==cid].iloc[0].Duration
    obj['Duration']=duration
    for v in versions:
        currC = resultsDF[resultsDF['ContestId']==cid]
        thisStr = currC[v].iloc[0]
        if(isinstance(thisStr, float)):
            print(thisStr)
        AB = thisStr[1:-1].split(', ')
#         print(AB)
        obj['A'+v]=AB[0]
        obj['B'+v]=AB[1]
    if(count%10000==0):
        print(str(count)+".../..."+str(len(cids)))
#         print(obj)
    combinedObjs.append(obj)

10000.../...187173
20000.../...187173
30000.../...187173
40000.../...187173
50000.../...187173
60000.../...187173
70000.../...187173
80000.../...187173
90000.../...187173
100000.../...187173
110000.../...187173
120000.../...187173
130000.../...187173
140000.../...187173
150000.../...187173
160000.../...187173
170000.../...187173
180000.../...187173


In [33]:
# cDF = pd.DataFrame(combinedObjs)

In [40]:
combinedDF = pd.DataFrame(data = combinedObjs).set_index('ContestId')

In [41]:
combinedDF.head()

,Av1,Av10,Av11,Av12,Av13,Av14,Av2,Av4,Av5,Av6,...,Bv11,Bv12,Bv13,Bv14,Bv2,Bv4,Bv5,Bv6,Bv7,Duration
ContestId,,,,,,,,,,,,,,,,,,,,,
57276615,1.0284784169702672,4.365322891225454,3.1329704821061712,4.266830421375789,4.315577537817956,1.0004515556411093,1.0318827228224832,2.8887272558919532,1.8700974412944842,3.8704562005644525,...,0.0020823533896226236,0.0015933178752466988,0.001579819845326505,0.004008760692452682,0.003953985402647066,0.002208287885636045,0.002924750407073967,0.0017505192506999205,0.004004137570674898,1124
57276616,1.07519393313605,10.817511590413538,5.187555323857137,16.347671824184154,11.248968200165791,1.001109849820683,1.0834468428594874,4.507335243256082,2.485163815001465,7.30337264469591,...,0.003820877902879531,0.002886929294761843,0.003191086287926681,0.005159490500272467,0.005095179378489776,0.003935244445128993,0.004419677265931545,0.0035425428603767,0.0051574197608539605,1473
57276617,1.0020358192733312,2.144953175476162,1.5388008137135483,2.224963631977283,2.2463668055319492,1.0000647795621955,1.002202786586186,1.4519719052080122,1.103485870186017,1.760274075577878,...,0.002848157986934873,0.0023410860290446257,0.002333667749343136,0.003436597725738158,0.003433669676831122,0.0029271379985731836,0.003301796154889165,0.0026652433286112422,0.0034329929591686522,1319
57276618,1.0619352955624248,14.140700099231703,8.22118316189748,20.080096987025534,14.414935414588754,1.002339645775642,1.0772112928174329,7.475440329016996,4.832088338433924,10.681851638544483,...,0.003668616280418197,0.0029496014251527004,0.0032164812611672373,0.005362966291076187,0.005304963965863303,0.0037451793217003608,0.004096503968137594,0.0034578010315305524,0.005361116371917111,1487
57276619,1.4935814614841791,3.9875329966948363,3.528179377963562,3.630030526567865,3.948903724726818,1.0121902097237188,1.5240505043053543,3.3954742746531887,2.775160687138497,3.866468076234614,...,0.001952426012757016,0.0019098083484540367,0.0017838951978086295,0.04443524570262858,0.008679691023508029,0.0020097888527249924,0.0023117469631557396,0.0018154597320470475,0.04406937555848803,943


In [42]:
combinedDF.to_csv('data/QR_Results.csv')

---

In [43]:
qrDF = pd.read_csv('data/QR_Results.csv').set_index('ContestId')

In [51]:
versions = []
for v in qrDF.columns:
    if(v[0]=="A"):
        versions.append(v[1:])

In [61]:
predCols = []
for v in versions:
    predCol = 'Pred'+v
    qrDF[predCol] = qrDF['A'+v] *np.exp(qrDF['B'+v]*qrDF['Duration'])
    predCols.append(predCol)
predDF = qrDF[predCols].dropna()

In [65]:
predDF.to_csv('data/QR_Preds.csv')

In [121]:
predDF.head()

,Predv1,Predv10,Predv11,Predv12,Predv13,Predv14,Predv2,Predv4,Predv5,Predv6,Predv7
ContestId,,,,,,,,,,,
57276615,88.140383,25.286467,32.542496,25.578715,25.481397,9.058589e+01,87.852969,34.568110,50.069398,27.686807,9.035033e+01
57276616,1972.120830,1247.019083,1442.907857,1148.867947,1237.373802,2.000273e+03,1969.129222,1483.739978,1669.906961,1348.167957,1.999262e+03
57276617,92.883431,50.584721,65.872681,48.795253,48.784953,9.303141e+01,92.870929,68.980068,85.930882,59.200233,9.283373e+01
57276618,2880.371200,1728.417409,1923.586218,1612.895749,1721.880843,2.913376e+03,2872.267259,1960.009650,2136.192127,1826.754779,2.912056e+03
57276619,24118.874113,21.150367,22.240606,21.981261,21.235009,1.596832e+18,5466.097649,22.593773,24.549366,21.419891,1.132696e+18


In [91]:
successDF = pd.read_csv('data/WorkedData.csv').set_index('ContestId')[['Success']]
contestsDF = pd.read_csv('data/WorkingData.csv').set_index('ContestId')[['MaxNumberPlayers']]
contestsDF = pd.merge(contestsDF, successDF, on='ContestId', how='left')

In [115]:
combinedDF = pd.merge(predDF, contestsDF, on='ContestId', how='left')
for v in versions:
    combinedDF["Pred"+v] = combinedDF["Pred"+v] / combinedDF["MaxNumberPlayers"]
combinedDF = combinedDF.drop(columns=['MaxNumberPlayers'])
combinedDF = combinedDF.replace(np.inf, 1.0).dropna()

In [118]:
combinedDF['AvgPred'] = combinedDF.mean(axis=1)

In [120]:
combinedDF.dropna().to_csv('data/QR_Preds_Scaled.csv')

,Predv1,Predv10,Predv11,Predv12,Predv13,Predv14,Predv2,Predv4,Predv5,Predv6,Predv7,Success
ContestId,,,,,,,,,,,,
57276615,3.039324e+00,8.719471e-01,1.122155e+00,8.820246e-01,8.786689e-01,3.123651e+00,3.029413e+00,1.192004e+00,1.726531e+00,9.547175e-01,3.115529e+00,True
57276616,1.658638e+00,1.048797e+00,1.213547e+00,9.662472e-01,1.040684e+00,1.682315e+00,1.656122e+00,1.247889e+00,1.404463e+00,1.133867e+00,1.681465e+00,True
57276618,2.422516e+00,1.453673e+00,1.617819e+00,1.356515e+00,1.448176e+00,2.450274e+00,2.415700e+00,1.648452e+00,1.796629e+00,1.536379e+00,2.449164e+00,True
57276620,1.334179e+00,9.859068e-01,1.142415e+00,9.567217e-01,9.785034e-01,1.334954e+00,1.334110e+00,1.172709e+00,1.291134e+00,1.058683e+00,1.334016e+00,True
57276621,1.571221e+00,9.911412e-01,1.203011e+00,9.353550e-01,9.825114e-01,1.573409e+00,1.570987e+00,1.255247e+00,1.466764e+00,1.089364e+00,1.572135e+00,True
57276624,1.473610e+00,9.362981e-01,1.104447e+00,8.464062e-01,9.305990e-01,1.497748e+00,1.471139e+00,1.147521e+00,1.267033e+00,1.018434e+00,1.496978e+00,True
57277130,1.861915e+00,1.532586e+00,1.691945e+00,1.449439e+00,1.529384e+00,1.862697e+00,1.861826e+00,1.730182e+00,1.822087e+00,1.608687e+00,1.862401e+00,True
57277201,1.060007e+00,7.046614e-01,8.158100e-01,6.499525e-01,6.998690e-01,1.068967e+00,1.058956e+00,8.374832e-01,9.329719e-01,7.607581e-01,1.068489e+00,True
57277202,8.082041e-01,6.298621e-01,6.954045e-01,6.551933e-01,6.324733e-01,8.085503e-01,8.081764e-01,7.272191e-01,7.903755e-01,6.502956e-01,8.080346e-01,True
